In [63]:
# 재시작 없이 import 파일 자동 적용
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [79]:
import os
import torch
import random
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch.utils.data import DataLoader
from data_loader import PublicTest
from data_factory import collate_fn
from config import CONFIG
from classification import Exp_Classification

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

In [80]:
seed = CONFIG['seed']
    
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [81]:
setting = "{}_{}_{}_{}_bs{}_sl{}_lr{}_pl{}_dm{}_nh{}_el{}_df{}_fc{}_eb{}_{}_seed{}".format(
                CONFIG['task_name'],
                CONFIG['model_id'],
                CONFIG['model'],
                CONFIG['data'],
                CONFIG['batch_size'],
                CONFIG['seq_len'],
                CONFIG['learning_rate'],
                CONFIG['pred_len'],
                CONFIG['d_model'],
                CONFIG['n_heads'],
                CONFIG['e_layers'],
                CONFIG['d_ff'],
                CONFIG['factor'],
                CONFIG['embed'],
                CONFIG['des'],
                CONFIG['seed'],
            )

exp = Exp_Classification(CONFIG)

Use GPU: cuda:0


In [82]:
testset = PublicTest("./dataset/KMedicon/public_test2.h5")
test_loader = DataLoader(
    testset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    drop_last=False,
    # collate_fn=lambda x: collate_fn(
    #             x, max_len=CONFIG['seq_len']),
    )

In [87]:
# model_path = "./checkpoints/classification_HM05_Medformer_K-Medicon_bs4_sl96_lr0.0001_pl96_dm128_nh8_el6_df256_fc1_ebtimeF_Exp_seed44/checkpoint.pth"
model_path = "./BestModels/JT08_2.pth"
exp.swa_model.load_state_dict(torch.load(model_path))
exp.swa_model.eval()

device = torch.device("cuda:0")

total_loss = []
preds = []
trues = []

with torch.no_grad():
    for i, (batch_x, label) in enumerate(tqdm(test_loader)):
        batch_x = batch_x.float().to(device)
        label = label.to(device)
        # padding_mask = padding_mask.float().to(device)

        outputs = exp.swa_model(batch_x, None, None, None)

        pred = outputs.detach().cpu()

        preds.append(outputs.detach())
        trues.append(label)

preds = torch.cat(preds, 0)
trues = torch.cat(trues, 0)

probs = torch.nn.functional.softmax(
    preds
)  # (total_samples, num_classes) est. prob. for each class and sample

trues_onehot = (torch.nn.functional.one_hot(trues.reshape(-1,).to(torch.long),
                num_classes=CONFIG['num_class'],).float().cpu().numpy())

# print(trues_onehot.shape)
predictions = (
    torch.argmax(probs, dim=1).cpu().numpy()
)  # (total_samples,) int class index for each sample
probs = probs.cpu().numpy()
trues = trues.flatten().cpu().numpy()

100%|██████████| 300/300 [00:10<00:00, 27.89it/s]


In [88]:
f1 = f1_score(trues, predictions, average="binary")
auroc = roc_auc_score(trues_onehot, probs)
mcc = matthews_corrcoef(trues, predictions)
cpi = (0.25 * f1) + (0.25 * auroc) + (0.5 * mcc)

print(f"F1 score: {f1}")
print(f"AUROC score: {auroc}")
print(f"MCC score: {mcc}")
print(f"CPI score: {cpi}")

F1 score: 0.6706586826347306
AUROC score: 0.8502518518518518
MCC score: 0.5509189647176966
CPI score: 0.6556871159804939


CPI score: 0.6741822771667003

In [89]:
print(f"0개수: {len(np.where(predictions[:]==0)[0])}")
print(f"1개수: {len(np.where(predictions[:]==1)[0])}")

0개수: 208
1개수: 92


In [90]:
import pandas as pd

df = pd.DataFrame({"possibility":probs[:,1]})

save_xlsx = pd.ExcelWriter("./public_results/RTFACT_241011_Public_Possibility_JT08_2_Softmax.xlsx")
df.to_excel(save_xlsx, index = False) # xlsx 파일로 변환

save_xlsx.save() #xlsx 파일로 저장

In [16]:
predictions[15]

1